## <font color="blue"> Data</font>

In [1]:
include("../src/JWAS.jl")
using DataFrames
using QTLDatasets
using JWAS.MT

In [2]:
phenofile = QTLDatasets.dataset("testMT","phenotype.txt")
genofile  = QTLDatasets.dataset("testMT","genotype.txt")
pedfile   = QTLDatasets.dataset("testMT","pedigree.txt");

### Genetic covariance matrix and residual covariance matrix

In [3]:
R      = [10.0 0.0
           0.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [4]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


## <font color="blue">3. mulitiple traits with marker effects</font> 

#### set up model equations

In [5]:
model_equations = "BW = intercept + age + sex;
                   CW = intercept + age + sex";

In [6]:
model3    = MT.build_model(model_equations,R);

In [7]:
MT.set_covariate(model3,"age")

In [8]:
MT.add_markers(model3,genofile,G,separator=',',header=true);

The delimiters in file /Users/haocheng/.julia/v0.4/QTLDatasets/src/../data/testMT/genotype.txt is ,  .


In [9]:
Pi=Dict([1.0; 1.0]=>0.25,[1.0;0.0]=>0.25,[0.0,1.0]=>0.25,[0.0; 0.0]=>0.25)
C = [false true;true false]
out2 = MT.runMCMC(model3,data,Pi=Pi,chain_length=50,constraint=C,printout_frequency=10,methods="BayesC",output_marker_effects_frequency=1);

LoadError: LoadError: UndefVarError: constrainta not defined
while loading In[9], in expression starting on line 3

In [10]:
using JWAS.QTL

In [11]:
file1="marker_effects_BW_2016-07-15T00:20:40.txt"
file2="marker_effects_CW_2016-07-15T00:20:40.txt";

In [12]:
a,b,c=get_additive_genetic_variances(model3,file1,file2);

In [13]:
get_breeding_values(model3,file1,file2)

2-element Array{Any,1}:
 4×3 DataFrames.DataFrame
│ Row │ ID   │ EBV      │ PEV     │
├─────┼──────┼──────────┼─────────┤
│ 1   │ "S1" │ -14.7203 │ 28.8167 │
│ 2   │ "D1" │ 5.74251  │ 34.8293 │
│ 3   │ "O1" │ 21.217   │ 48.5628 │
│ 4   │ "O3" │ -12.2392 │ 18.5602 │
 4×3 DataFrames.DataFrame
│ Row │ ID   │ EBV      │ PEV     │
├─────┼──────┼──────────┼─────────┤
│ 1   │ "S1" │ -2.94535 │ 2.39471 │
│ 2   │ "D1" │ 1.56458  │ 1.95413 │
│ 3   │ "O1" │ 3.70722  │ 4.90648 │
│ 4   │ "O3" │ -2.32645 │ 1.21073 │